**1. Generation of Feasible Datasets for the Transmission Channel Parameters**

In [ ]:
import numpy as np

## Number of transmitter-receiver pairs
K = 5

## Variances for noise signals
sigma_sqr_noise = np.array([1e-0, 1e-0, 1e-0, 1e-0, 1e-0], dtype = float)

## Minimum rate for the achievable SINR of multiple concurrent transmissions
SINR_P_min = np.array([0.5, 0.5, 0.5, 0.5, 0.5], dtype = float)

## Maximum transmit power
p_max = 1.0

In [ ]:
## Function to generate the Circularly Symmetric Complex Gaussian (CSCG) distributions

def complex_gaussian(d_mean = 0, d_var = 1, n = 1000):
  # Draw random samples from a normal (Gaussian) distribution.
  # Parameters:
  #   loc = Mean (“centre”) of the distribution.
  #   scale = Standard deviation (spread or “width”) of the distribution. Must be non-negative.
  #   size = int or tuple of ints, optional
  return np.random.normal(loc = d_mean, scale = np.sqrt(2*d_var)/2, size = (n, 2)).view(np.complex128)


## Function to generate the channel-coefficient matrix H
def generate_H(K, sigma_sqr_h, sample_size):
  hij = []
  for i in range(K):  # Total rows, i.e., total receivers or users
    hj =[]
    for j in range(K):  # Total columns, i.e., total transmitters
      h = complex_gaussian(d_mean = 0, d_var = sigma_sqr_h, n = sample_size)
      hj.append(h)
    hj = np.concatenate(hj, 1)
    hij.append(hj)
  hij = np.stack(hij, 1)
  return hij

In [ ]:
## Create matrix H
H_size = int(1e6)
sigma_sqr_h = 1
# np.random.seed(0)
H = generate_H(K, sigma_sqr_h, H_size)
print(H.shape)
# print(H)

(1000000, 5, 5)


In [ ]:
import numba as nb

## Function to compute the square of the absolute value of an array of complex numbers
@nb.vectorize([nb.float64(nb.complex128),nb.float32(nb.complex64)])
def cmplx_abs_sqr(cmplx_var):
  return cmplx_var.real**2 + cmplx_var.imag**2

In [ ]:
## Function to generate the matrix B
def generate_B(H_size, K, SINR_P_min, H):
  Bij_list = []
  H_abs_sqr = cmplx_abs_sqr(H)
  for k in range(H_size):
    for i in range(K):  # Total rows
      Bj_list =[]
      for j in range(K):  # Total columns
        if i==j:
          B = 0
        else:
          B_temp = np.multiply(SINR_P_min[i], H_abs_sqr[k,i,j])
          B = np.divide(B_temp, H_abs_sqr[k,i,i])
        Bj_list.append(B)
      Bij_list.append(Bj_list)
  Bij_array = np.array(Bij_list)
  Bij = Bij_array.reshape((H_size, K, K)) # H_size X row X column
  return Bij

In [ ]:
## Create matrix B
B = generate_B(H_size, K, SINR_P_min, H)
print(B.shape)
# print(B)

(1000000, 5, 5)


In [ ]:
## Function to generate the vector u
def generate_u(H_size, K, SINR_P_min, sigma_sqr_noise, H):
  ui_list = []
  H_abs_sqr = cmplx_abs_sqr(H)
  for k in range(H_size):
    for i in range(K):  # Total rows, i.e., total transmitters
      u_temp = np.multiply(SINR_P_min[i], sigma_sqr_noise[i])
      u = np.divide(u_temp, H_abs_sqr[k,i,i])
      ui_list.append(u)
  ui_array = np.array(ui_list)
  ui = ui_array.reshape((H_size, K, 1)) # H_size X row X column
  return ui

In [ ]:
## Create vector u
u = generate_u(H_size, K, SINR_P_min, sigma_sqr_noise, H)
print(u.shape)
# print(u)

(1000000, 5, 1)


In [ ]:
## Finding indexes of H matrix with the hij set that satisfy constraint on the
## maximum transmit power p_max

count_var = 0
indx_F_H = []
indx_temp_F_H = []
p_hat_temp_list = []

for k in range(H_size):
  eigen_value, eigen_vector = np.linalg.eig(B[k])
  # print(eigen_value)
  if max(abs(eigen_value)) < 1:
    subtr = np.identity(K) - B[k,:,:]
    invr = np.linalg.inv(subtr)
    u_temp = u[k]
    p_temp = np.matmul(invr, u_temp)
    p_hat_temp_list.append(p_temp)
    indx_temp_F_H.append(k)
    count_var += 1

p_hat_temp_array = np.array(p_hat_temp_list)
p_hat_temp = p_hat_temp_array.reshape((count_var,K,1))
print(p_hat_temp.shape)
# print(p_hat_temp)


P = abs(p_hat_temp)
fcount = 0
p_hat_list = []
for n in range(count_var):
  P_max = np.amax(P[n])
  if P_max <= p_max:
    p = p_hat_temp[n]
    p_hat_list.append(p)
    indx_F_H.append(indx_temp_F_H[n])
    fcount += 1

p_hat_array = np.array(p_hat_list)
p_hat = p_hat_array.reshape((fcount,K,1))
# p_hat = p_hat_array.reshape((fcount,1,K))
print(p_hat.shape)
p_hat_size = p_hat.shape[0]
# print(p_hat)

(11044, 5, 1)
(317, 5, 1)


In [ ]:
## H matrix for a feasible power profile
F_H_size = len(indx_F_H)
F_H = np.empty((F_H_size, K, K), dtype = complex, order = 'C')

for i in range(F_H_size):
  j = indx_F_H[i]
  F_H[i] = H[j]

print(F_H.shape)
# print(F_H)

(317, 5, 5)


In [ ]:
# ## Checking SINR_P for feasible H matrix
# F_H_abs_sqr = cmplx_abs_sqr(F_H)

# for k in range(F_H_size):
#   SINR_P_F_H_list = []
#   for i in range(K):
#     ph = 0
#     for j in range(K):
#       ph_j = np.multiply(p_hat[k,j], F_H_abs_sqr[k,i,j])
#       ph = ph + ph_j

#     numr = np.multiply(p_hat[k,i], F_H_abs_sqr[k,i,i])
#     dnumr = sigma_sqr_noise[i] + ph - numr
#     SINR_P_temp = np.divide(numr, dnumr)
#     SINR_P_F_H_list.append(SINR_P_temp)

#   SINR_P_F_H_array = np.array(SINR_P_F_H_list)
#   SINR_P_F_H = SINR_P_F_H_array.reshape((1, K))
#   print(SINR_P_F_H)
#   p_hat_t = p_hat[k].reshape((1, 1, K)) # H_size X row X column
#   print(p_hat_t)

In [ ]:
# ## Saving 3D Numpy array to CSV file
# # Saving feasible H matrix F_H
# from numpy import savetxt

# # Reshaping the array from 3D to 2D
# F_H_2D = F_H.reshape(F_H.shape[0], -1)

# # Saving reshaped array to file in "Files" of colab at left bar
# # Can download the file in local drive
# savetxt('F_H_2D.csv', F_H_2D, delimiter=',')

In [ ]:
# ## Saving p_hat matrix to CSV file
# # from numpy import savetxt

# # Reshaping the array from 3D to 2D
# p_hat_2D = p_hat.reshape(p_hat.shape[0], -1)

# # Saving reshaped array to file in "Files" of colab at left bar
# # Can download the file in local drive
# savetxt('p_hat_2D.csv', p_hat_2D, delimiter=',')

**Codes to calculate the average sum rate for the basic model**

In [ ]:
## Function to split datasets for training, validation, and testing.
def split(np_array):
  # data_size = np_array.shape[0]
  # train_data_size = int(data_size * 0.8)
  # valid_data_size = int(data_size * 0.1)
  # test_data_size = int(data_size * 0.1)

  train_data_size = int(200000)
  valid_data_size = int(25000)
  test_data_size = int(25000)

  train_e_indx = train_data_size
  valid_e_indx = train_e_indx + valid_data_size
  test_e_indx = valid_e_indx + test_data_size
  test_data_size_n = test_e_indx - valid_e_indx

  row_count = np_array.shape[1]
  column_count = np_array.shape[2]

  train_data = np.empty((train_data_size, row_count, column_count), dtype = complex, order = 'C')
  valid_data = np.empty((valid_data_size, row_count, column_count), dtype = complex, order = 'C')
  test_data = np.empty((test_data_size_n, row_count, column_count), dtype = complex, order = 'C')

  for i in range(train_e_indx):
    train_data[i] = np_array[i]

  xv = 0
  for j in range(train_e_indx, valid_e_indx):
    valid_data[xv] = np_array[j]
    xv = xv + 1

  xt = 0
  for k in range(valid_e_indx, test_e_indx):
    test_data[xt] = np_array[k]
    xt = xt + 1

  # print(train_data.shape, valid_data.shape, test_data.shape)


  ## Training input will be the absolute value
  train_input = np.absolute(train_data)
  valid_input = np.absolute(valid_data)
  test_input = np.absolute(test_data)

  print(train_input.shape, valid_input.shape, test_input.shape)

  return [train_input, valid_input, test_input, test_data]

In [ ]:
## Split F_H matrix
F_H_S = split(F_H)
train_input_F_H = F_H_S[0]
valid_input_F_H = F_H_S[1]
test_input_F_H = F_H_S[2]
test_data_F_H = F_H_S[3]

In [ ]:
## Split p_hat vector
p_hat_S = split(p_hat)
train_input_p_hat = p_hat_S[0]
valid_input_p_hat = p_hat_S[1]
test_input_p_hat = p_hat_S[2]
test_data_p_hat = p_hat_S[3]

In [ ]:
## Function to calculate the average sum rate
# Here, p_model is the output of DNN, and it is a 2D array.
import math

def average_sum_rate(hij, p_model, sigma_sqr_noise, K):
  R = 0
  hij_size = hij.shape[0]
  hij_abs_sqr = cmplx_abs_sqr(hij)

  for k in range(hij_size):
    for i in range(K):  # Total rows
      phn = 0
      for j in range(K):  # Total columns
        phn_j = np.multiply(p_model[k,j], hij_abs_sqr[k,i,j])
        phn = phn + phn_j

      numr_s = np.multiply(p_model[k,i], hij_abs_sqr[k,i,i])
      dnumr_s = sigma_sqr_noise[i] + phn - numr_s
      R_temp = math.log2(1 + np.divide(numr_s, dnumr_s))
      R = R + R_temp

  return (R/hij_size)

In [ ]:
# DNN Sum Rate for test_data_F_H
output_P_hat = abs(test_data_p_hat)
sumrate_F_H = average_sum_rate(test_data_F_H, output_P_hat, sigma_sqr_noise, K)
print("Average Sum Rate for all H matrices: {:.3f} Bit/Second/Hertz".format(sumrate_F_H))